## Marc Gosine

# Importing Data From Python

In [2]:
college = read.csv('scorecard.csv')
#importing dataset. Originally assembled in Python
college = na.omit(college)
#omitting NA and NULLs to avoid errors in regression. Code from R for Dummies
names(college)
#Showing column names

[1] "school.region_id"                                
[2] "latest.admissions.sat_scores.average.overall"    
[3] "school.tuition_revenue_per_fte"                  
[4] "school.name"                                     
[5] "school.ft_faculty_rate"                          
[6] "school.faculty_salary"                           
[7] "latest.student.demographics.race_ethnicity.white"

# Data Cleaning

The dataset imported into R did not contain the unique variable or logged variable since that was created afterwards specifically for the scatterplots later on. For this file, I created the salary to revenue ratio, I filtered out Inf variables to avoid errors when running regressions (some nubers were divded by zero, creating Inf variables) and I renamed columns to more concise names. Above, I also omitted NAs because I was unable to run regressions with them.

In [3]:
#Extra Data Cleaning
library('dplyr')
college2 = mutate(college, salary_to_rev = round(school.faculty_salary/school.tuition_revenue_per_fte,digits = 3))
#Creating unique variable which is the ratio of the faculty salary and tution revenue per full time student
college2 = filter(college2, !is.infinite(salary_to_rev))
#Getting rid of Inf values to avoid errors when running regression. Code from Quick R
college2 = rename(college2, SAT_Scores = latest.admissions.sat_scores.average.overall, Full_Time_Rate = school.ft_faculty_rate, 
       Percent_White = latest.student.demographics.race_ethnicity.white)
#Renaming colums to make names more comprehensible


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



# Regression

My final regression runs robust standard errors with three independent variables which were selected to describe the average SAT score in a given university. The idea behind this regression was to find out how a given set of features affect the average SAT Score in a college. In other words, how do colleges attract people with a higher score?

The first variable I chose was the full-time faculty rate (Percentage of people under full time employment). My rationale for this variable is that the full-time faculty rate could be a good selling point to attract better performing students. The final model shows that a one percentage point increase (expressed as 0.01 since the column is a decimal) on average leads to a school having a higher SAT average by about 1.29. In both models, this variable is statistically significant at 1%. In other words, we are 99% confident that the true value in within range of the confidence interval.

The second variable I chose to do was the region of the school. My rationale for including this was because the reputation of the school can vary by region. For example, New England has the ivy league schools. Also, wealthier regions may be able to better fund their schools with taxes or subsidies from state governments. Each categorical variable represented a region in the United States. 1 represented New England, 2 represented the Mid-East, 3 represented the Great Lakes region, etc. The model that was run without robust standard errors showed all of these variables to be all statistically insignificant. However, the robust model made all of these variables statistically significant at <1%. To interpret these coefficients, a school in a given region on average has a B1 decrease in their SAT score average. Rather than going through the painstaking process of interpreting each one, I will list what I found notable. All of the dummy variables had a negative sign, which I found intriguing but two regions stood out to me: 1 and 8 (AKA New England and the West Coast). Their coefficients were the lowest in magnitude at 64 and 98 respectively. So, I interpret these areas as the regions with the highest scores relative to the others on average. Basically, the coefficients cannot be intetrpreted on their own but they can be interpreted in comparison to others.

The third and final variable is salary to revenue, which I created by dividing the average faculty salary by tuition revenue per full time student. My rationale for doing this was to find out how much was of revenue generated by tuition was being invested in faculty. A higher salary relative to revenue may be more attractive to a student and may cause a college to raise the bar of their SAT score criteria. In the non robust model, the variable is statistically significant at 1%. The robust model drops this significance to 5%. A 1% increase in this ratio translates to a 22 point decrease on average.

In [5]:
r3 = lm(SAT_Scores ~ log(salary_to_rev) + Full_Time_Rate 
        + factor(school.region_id), data = college2)
#Initial Preferred Regression (No robust std errors) 
summary(r3)
#Calls upon regression analysis


Call:
lm(formula = SAT_Scores ~ log(salary_to_rev) + Full_Time_Rate + 
    factor(school.region_id), data = college2)

Residuals:
    Min      1Q  Median      3Q     Max 
-367.74  -75.77  -16.86   66.16  515.21 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)               1206.885    119.836  10.071  < 2e-16 ***
log(salary_to_rev)         -22.183      5.571  -3.982 7.23e-05 ***
Full_Time_Rate             129.339     14.397   8.984  < 2e-16 ***
factor(school.region_id)1  -98.069    119.875  -0.818    0.413    
factor(school.region_id)2 -144.533    119.426  -1.210    0.226    
factor(school.region_id)3 -170.485    119.427  -1.428    0.154    
factor(school.region_id)4 -166.579    119.546  -1.393    0.164    
factor(school.region_id)5 -191.809    119.313  -1.608    0.108    
factor(school.region_id)6 -196.378    119.752  -1.640    0.101    
factor(school.region_id)7 -157.763    121.051  -1.303    0.193    
factor(school.region_id)8 -123.438 

In [6]:
library(estimatr)
#loaded in to run robust model
r4 = lm_robust(SAT_Scores ~ log(salary_to_rev) + Full_Time_Rate 
        + factor(school.region_id), data = college2)
#Finalized Regression (With robust standard errors to solve for heteroskedasticity)
summary(r4)


Call:
lm_robust(formula = SAT_Scores ~ log(salary_to_rev) + Full_Time_Rate + 
    factor(school.region_id), data = college2)

Standard error type:  HC2 

Coefficients:
                          Estimate Std. Error t value   Pr(>|t|) CI Lower
(Intercept)                1206.88     14.694  82.135  0.000e+00  1178.06
log(salary_to_rev)          -22.18     11.383  -1.949  5.154e-02   -44.52
Full_Time_Rate              129.34     14.203   9.107  3.208e-19   101.48
factor(school.region_id)1   -98.07     17.157  -5.716  1.358e-08  -131.73
factor(school.region_id)2  -144.53      9.896 -14.605  8.874e-45  -163.95
factor(school.region_id)3  -170.48      7.946 -21.456  2.116e-87  -186.07
factor(school.region_id)4  -166.58      7.959 -20.928  8.346e-84  -182.19
factor(school.region_id)5  -191.81      7.259 -26.424 5.416e-123  -206.05
factor(school.region_id)6  -196.38     12.093 -16.239  4.974e-54  -220.10
factor(school.region_id)7  -157.76     17.173  -9.186  1.604e-19  -191.45
factor(school.reg

# Regression Table

This regression table contains two full models. The first model is run without robust standard errors while the second is run with them. In order to accomplish this, I had to run the same model twice but in the "se" argument, I had to specify that the second regression must use the statisical adjustments vreated under 'robust_se'.The code for these statisical adjustments came from https://www.jakeruss.com/cheatsheets/stargazer/

In [8]:
library(estimatr)
library(stargazer)
library(lmtest)  
library(sandwich)

cov1         = vcovHC(r3, type = "HC1")
robust_se    = sqrt(diag(cov1))
# Adjust standard errors

wald_results = waldtest(r3, vcov = cov1)
# Adjust F statistic 

stargazer(r3, r3, type = "html", column.labels = c("Normal","Robust"), se = list(NULL, robust_se),
          omit.stat = "f", model.names = FALSE,
          title            = "Data Results",
          covariate.labels = c("Salary to Revenue (logged)", "Full Time Rate","New England","Mid East","Great Lakes","Plains","Southeast",
                              "Southwest","Rocky Mountains","West Coast","Territories"),
          dep.var.labels   = "SAT Scores")
#Code for regression table


<table style="text-align:center"><caption><strong>Data Results</strong></caption>
<tr><td colspan="3" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="2"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="2" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="2">SAT Scores</td></tr>
<tr><td style="text-align:left"></td><td>Normal</td><td>Robust</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td></tr>
<tr><td colspan="3" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">Salary to Revenue (logged)</td><td>-22.183<sup>***</sup></td><td>-22.183<sup>**</sup></td></tr>
<tr><td style="text-align:left"></td><td>(5.571)</td><td>(10.731)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">Full Time Rate</td><td>129.339<sup>***</sup></td><td>129.339<sup>***</sup></td></tr>
<tr><td styl

<table style="text-align:center"><caption><strong>Data Results</strong></caption>
<tr><td colspan="3" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="2"><em>Dependent variable:</em></td></tr>
<tr><td></td><td colspan="2" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td colspan="2">SAT Scores</td></tr>
<tr><td style="text-align:left"></td><td>Normal</td><td>Robust</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td></tr>
<tr><td colspan="3" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">Salary to Revenue (logged)</td><td>-22.183<sup>***</sup></td><td>-22.183<sup>**</sup></td></tr>
<tr><td style="text-align:left"></td><td>(5.571)</td><td>(10.731)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">Full Time Rate</td><td>129.339<sup>***</sup></td><td>129.339<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(14.397)</td><td>(14.178)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">New England</td><td>-98.069</td><td>-98.069<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(119.875)</td><td>(17.125)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">Mid East</td><td>-144.533</td><td>-144.533<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(119.426)</td><td>(9.908)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">Great Lakes</td><td>-170.485</td><td>-170.485<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(119.427)</td><td>(7.958)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">Plains</td><td>-166.579</td><td>-166.579<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(119.546)</td><td>(7.959)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">Southeast</td><td>-191.809</td><td>-191.809<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(119.313)</td><td>(7.234)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">Southwest</td><td>-196.378</td><td>-196.378<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(119.752)</td><td>(12.053)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">Rocky Mountains</td><td>-157.763</td><td>-157.763<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(121.051)</td><td>(16.940)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">West Coast</td><td>-123.438</td><td>-123.438<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(119.686)</td><td>(12.538)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">Territories</td><td>-218.344</td><td>-218.344<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(138.297)</td><td>(34.840)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td style="text-align:left">Constant</td><td>1,206.885<sup>***</sup></td><td>1,206.885<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(119.836)</td><td>(14.468)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td></tr>
<tr><td colspan="3" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">Observations</td><td>1,283</td><td>1,283</td></tr>
<tr><td style="text-align:left">R<sup>2</sup></td><td>0.106</td><td>0.106</td></tr>
<tr><td style="text-align:left">Adjusted R<sup>2</sup></td><td>0.098</td><td>0.098</td></tr>
<tr><td style="text-align:left">Residual Std. Error (df = 1271)</td><td>119.120</td><td>119.120</td></tr>
<tr><td colspan="3" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"><em>Note:</em></td><td colspan="2" style="text-align:right"><sup>*</sup>p<0.1; <sup>**</sup>p<0.05; <sup>***</sup>p<0.01</td></tr>
</table>